In [1]:
!pip install delta-spark==1.1.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 7.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.2.4-py2.py3-none-any.whl size=282040922 sha256=1b0c5abedf166228d3fe2f814de6eec1443664b316abe6d1a5eb049c33c11a98
  Stored in directory: /home/jovyan/.cache/pip/wheels/bf/f6/18/acaa11d057c23e749eea1773c9516d58fbc8ddcdd1492ad3fd
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.3.0
    Can't uninstall 'pyspark'. No files were found to uninstall.
  Attempting uninstall: delta-spark
    Found existing installation: delta-spark 2.3.0
    Uninstalling delta-spark-2.3.0:
      Successfully uninstalled delta-spark-2.3.0


In [2]:
!pip install delta-spark


  Using cached pyspark-3.2.4-py2.py3-none-any.whl
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.3.0
    Can't uninstall 'pyspark'. No files were found to uninstall.


In [3]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from IPython.core.display import HTML
from pyspark.sql import functions as F
from delta import *

display(HTML("<style>pre { white-space: pre !important; }</style>"))

spark = (SparkSession.builder
         .config("spark.jars","""/home/jovyan/jars/aws-java-sdk-core-1.11.534.jar,
                                 /home/jovyan/jars/aws-java-sdk-dynamodb-1.11.534.jar,
                                 /home/jovyan/jars/aws-java-sdk-s3-1.11.534.jar,
                                 /home/jovyan/jars/hadoop-aws-3.2.2.jar,
                                 /home/jovyan/jars/postgresql-42.3.3.jar""")
         .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
         .config("spark.hadoop.fs.s3a.access.key", "ZiVYnHFvBt2HumCAXmQG")
         .config("spark.hadoop.fs.s3a.secret.key", "oVNgBEZ5PISLiwF25fJkoazHGXRFB7mJeiBcVygd")
         .config("spark.hadoop.fs.s3a.path.style.access", True)
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
         .getOrCreate()
        )

In [4]:
# Definindo o diretório de origem e o diretório de destino
input_path_format = "s3a://bronze/yellow_taxi_files/yellow_tripdata_2023-{month:02d}.parquet"
output_path = "s3a://silver/prd_yellow_taxi_table"

# Loop para iterar sobre os meses de 01 a 05
for month in range(1, 6):
    # Formatando o caminho do arquivo para o mês atual
    parquet_path = input_path_format.format(month=month)

    # Carregar os arquivos Parquet
    df = spark.read.parquet(parquet_path)

    # Caso haja discrepâncias de tipo entre os arquivos, faça o cast necessário
    df = (
        df
        .withColumn("VendorID", F.col("VendorID").cast(IntegerType()))
        .withColumn("tpep_pickup_datetime", F.col("tpep_pickup_datetime").cast(TimestampType()))
        .withColumn("tpep_dropoff_datetime", F.col("tpep_dropoff_datetime").cast(TimestampType()))
        .withColumn("passenger_count", F.col("passenger_count").cast(IntegerType()))
        .withColumn("trip_distance", F.col("trip_distance").cast(DoubleType()))
        .withColumn("RatecodeID", F.col("RatecodeID").cast(IntegerType()))
        .withColumn("store_and_fwd_flag", F.col("store_and_fwd_flag").cast(StringType()))
        .withColumn("PULocationID", F.col("PULocationID").cast(IntegerType()))
        .withColumn("DOLocationID", F.col("DOLocationID").cast(IntegerType()))
        .withColumn("payment_type", F.col("payment_type").cast(LongType()))
        .withColumn("fare_amount", F.col("fare_amount").cast(DoubleType()))
        .withColumn("extra", F.col("extra").cast(DoubleType()))
        .withColumn("mta_tax", F.col("mta_tax").cast(DoubleType()))
        .withColumn("tip_amount", F.col("tip_amount").cast(DoubleType()))
        .withColumn("tolls_amount", F.col("tolls_amount").cast(DoubleType()))
        .withColumn("improvement_surcharge", F.col("improvement_surcharge").cast(DoubleType()))
        .withColumn("total_amount", F.col("total_amount").cast(DoubleType()))
        .withColumn("congestion_surcharge", F.col("congestion_surcharge").cast(DoubleType()))
        .withColumn("airport_fee", F.col("airport_fee").cast(DoubleType()))
    )

    # Selecionando apenas as colunas necessárias
    df = (
        df.select(
        'VendorID',
        'passenger_count',
        'total_amount',
        'tpep_pickup_datetime',
        'tpep_dropoff_datetime'
        )
        .filter(
                (F.year(df["tpep_pickup_datetime"]) == 2023) & 
                (F.month(df["tpep_pickup_datetime"]).between(1, 5))
            )
      )

    # Exibindo o DataFrame para conferir
    df.show()

    # Salvando os dados no formato Delta no path final com append
    (
        df
        .write
        .format('delta')
        .mode('append')  # Usando append para adicionar ao Delta Lake
        .save(output_path)
    )


+--------+---------------+------------+--------------------+---------------------+
|VendorID|passenger_count|total_amount|tpep_pickup_datetime|tpep_dropoff_datetime|
+--------+---------------+------------+--------------------+---------------------+
|       2|              1|        14.3| 2023-01-01 00:32:10|  2023-01-01 00:40:36|
|       2|              1|        16.9| 2023-01-01 00:55:08|  2023-01-01 01:01:27|
|       2|              1|        34.9| 2023-01-01 00:25:04|  2023-01-01 00:37:49|
|       1|              0|       20.85| 2023-01-01 00:03:48|  2023-01-01 00:13:25|
|       2|              1|       19.68| 2023-01-01 00:10:29|  2023-01-01 00:21:19|
|       2|              1|        27.8| 2023-01-01 00:50:34|  2023-01-01 01:02:52|
|       2|              1|       20.52| 2023-01-01 00:09:22|  2023-01-01 00:19:49|
|       2|              1|       64.44| 2023-01-01 00:27:12|  2023-01-01 00:49:56|
|       2|              1|       28.38| 2023-01-01 00:21:44|  2023-01-01 00:36:40|
|   

In [5]:
from pyspark.sql.types import *
from pyspark.sql import functions as F
from delta import *

df_delta = spark.read.format("delta").load('s3a://silver/prd_yellow_taxi_table')

df_delta.show()


+--------+---------------+------------+--------------------+---------------------+
|VendorID|passenger_count|total_amount|tpep_pickup_datetime|tpep_dropoff_datetime|
+--------+---------------+------------+--------------------+---------------------+
|       1|              2|         9.4| 2023-02-01 00:32:53|  2023-02-01 00:34:34|
|       2|              1|        -5.5| 2023-02-01 00:35:16|  2023-02-01 00:35:30|
|       2|              1|         5.5| 2023-02-01 00:35:16|  2023-02-01 00:35:30|
|       1|              0|       74.65| 2023-02-01 00:29:33|  2023-02-01 01:01:38|
|       2|              1|        25.3| 2023-02-01 00:12:28|  2023-02-01 00:25:46|
|       1|              1|       32.25| 2023-02-01 00:52:40|  2023-02-01 01:07:18|
|       1|              1|        50.0| 2023-02-01 00:12:39|  2023-02-01 00:40:36|
|       1|              1|       14.64| 2023-02-01 00:56:53|  2023-02-01 01:00:37|
|       2|              1|       44.12| 2023-02-01 00:20:40|  2023-02-01 00:33:56|
|   

In [6]:
df_delta.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)



In [7]:
result = (
    df_delta.groupBy(F.year("tpep_pickup_datetime").alias("year"), F.month("tpep_pickup_datetime").alias("month"))
    .agg(F.sum("total_amount").alias("total_amount_sum"))
    .withColumn("total_amount_sum", F.format_number("total_amount_sum", 2))
    .orderBy("year", "month")
)

# Exibindo o resultado
result.show()

+----+-----+----------------+
|year|month|total_amount_sum|
+----+-----+----------------+
|2023|    1|   82,863,594.11|
|2023|    2|   78,381,600.43|
|2023|    3|   94,634,027.66|
|2023|    4|   92,956,843.11|
|2023|    5|  101,765,282.92|
+----+-----+----------------+

